In [1]:
from typing import Annotated, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

# SQL 쿼리 생성용
model_low_temp = ChatOllama(model="gemma3:1b", temperature=0.1)
# 자연어 출력 생성용
model_high_temp = ChatOllama(model="gemma3:1b", temperature=0.7)

/home/anseh/.pyenv/versions/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    user_query: str
    sql_query: str
    sql_explanation: str


class Input(TypedDict):
    user_query: str


class Output(TypedDict):
    sql_query: str
    sql_explanation: str

In [3]:
generate_prompt = SystemMessage(
    "당신은 친절한 데이터 분석가입니다. 사용자의 질문을 바탕으로 SQL쿼리를 작성하세요."
)

In [4]:
def generate_sql(state: State) -> State:
    user_message = HumanMessage(state["user_query"])
    messages = [generate_prompt, *state["messages"], user_message]
    res = model_low_temp.invoke(messages)
    return {"sql_query": res.content, "messages": [user_message, res]}

In [5]:
explain_prompt = SystemMessage(
    "당신은 친절한 데이터 분석가입니다. 사용자에게 SQL쿼리를 설명하세요."
)

In [6]:
def explain_sql(state: State) -> State:
    messages = [explain_prompt, *state["messages"]]
    res = model_high_temp.invoke(messages)
    return {"sql_explanation": res.content, "messages": res}

In [7]:
builder = StateGraph(State, input_schema=Input, output_schema=Output)
builder.add_node("generate_sql", generate_sql)
builder.add_node("explain_sql", explain_sql)
builder.add_edge(START, "generate_sql")
builder.add_edge("generate_sql", "explain_sql")
builder.add_edge("explain_sql", END)

graph = builder.compile()

In [8]:
result = graph.invoke({"user_query": "각 품목의 판매량을 구해주세요."})
print(result)

{'sql_query': '알겠습니다. 어떤 데이터가 가지고 있는지 알려주시면 SQL 쿼리를 작성해 드리겠습니다. \n\n데이터가 어떤 형태로 제공되어 있는지 (예: CSV 파일, 데이터베이스 테이블, 스프레드시트 등) 알려주세요. \n\n데이터를 제공해주시면, 다음과 같은 정보를 추출하여 SQL 쿼리를 작성해 드리겠습니다.\n\n*   **판매량:** 각 품목별 판매량\n*   **품목:** 품목 이름\n*   **판매일:** 판매일 (선택 사항)\n*   **판매처:** 판매처 (선택 사항)\n\n데이터를 제공해주시면, 필요한 SQL 쿼리를 작성해 드리겠습니다.', 'sql_explanation': "\n\n**예시:**\n\n만약 CSV 파일에 다음과 같은 데이터가 있다고 가정해 보겠습니다.\n\n```csv\n품목,판매일,판매량\n제품A,2023-10-26,100\n제품B,2023-10-26,50\n제품A,2023-10-27,120\n제품C,2023-10-27,80\n```\n\n위와 같은 데이터를 사용하면 다음과 같은 SQL 쿼리를 작성할 수 있습니다.\n\n```sql\nSELECT 품목, 판매량\nFROM 데이터\nWHERE 판매일 = '2023-10-26'\nORDER BY 판매량 DESC;\n```\n\n**데이터를 알려주시면 위와 같은 쿼리를 작성해 드리겠습니다.**"}
